# Basics of BigDFT: first runs and managing different calculations, N2 molecule as example

With this lesson you will have to deal with the different outputs of BigDFT code, such as to learn how to manipulate basic DFT objects.
We will use python and notebook to handle the input variables and run the calculations. The advantage is that you can automatize easily the calculations. It is also possible to run in a shell all these calculations.

First you need to have the correct environment variables: the script "bigdftvars.sh" from the install/bin directory of bigdft has to be sourced ("source bigdftvars.sh") before doing this tutorial and load the module python *BigDFT*.

In [1]:
from BigDFT import Calculators as calc

The main executable of the BigDFT suite is *bigdft*.

## Atomic position file: 'posinp.xyz'

A simple file with the position of atoms is enough to run bigdft performing a single-point calculation (calculating the ground state energy of the system). 
Consider for example the N2 molecule, given by the <a href="http://bigdft.org/images/f/f1/N2_posinp.xyz">posinp.xyz</a> file:
<pre>
2 angstroem
free
N 0. 0. 0.
N 0. 0. 1.11499</pre>
Copy it into the current directory.

In [2]:
study = calc.SystemCalculator() #Create a calculator
study.run() #run the code

Initialize a Calculator with OMP_NUM_THREADS= 1
used command /local/deutsch/Forge/BigDFT/build-mpif90/install/bin/bigdft
Executing command:  /local/deutsch/Forge/BigDFT/build-mpif90/install/bin/bigdft


<BigDFT.Logfiles.Logfile instance at 0x7f5e96fba7a0>

You can see exactly what the command is performed (you can do it in a bash shell).

Different files are created:
* [input_minimal.yaml](./input_minimal) which contains the minimal set of input variables to perform the run;
* [log.yaml](./log.yaml) which contains the log of the run with all calculated quantities;
* [time.yaml](./time.yaml) and [forces_posinp.xyz](./forces_posinp.xyz) which we will see later.

*bigdft* uses the yaml format which is a serialisation of the dict structure used in python. 
If you look at the [input_minimal.yaml](./input_minimal) file, you can see:

In this case, only the atomic positions are indicated using a yaml format.

In the log file [log.yaml](./log.yaml), BigDFT automatically displays all the input parameters used for the calculations. 

The parameters which were not explicitly given are set to a default value, except the atomic positions of course, which has to be given. Since they do not exist, their default values are applied to the code. Basically, they correspond to a single-point LDA calculation, without k-points nor spin-polarisation.
Because the format is the same, it is possible to use the 'log.yaml' file as input file

## Using a naming scheme for Input/Output files

All input parameters can be found in a yaml file with a naming prefix. By default, this prefix is *input* (or *posinp.xyz* for atomic input positions). One can choose the naming prefix by providing an argument to bigdft command line.

Upon launching, BigDFT will look by default for those input (and posinp) files to see which parameters it should use, but one can also decide to rename all those files with a particular name for more practicity. One then has to give this name when launching, so that BigDFT knows what to look for.
Imagine for example that you are interested in visualizing the wavefunctions output of the calculation. To do that, you should enter the suitable parameters in the .yaml file. 

Let us do it! Create a new calculation set by using the "LDA" prefix and rename all relevant files with the LDA prefix.
Instead of creating a yaml input file *LDA.yaml* by hand, we use the power of python creating a dictionay which will be serialized into yaml by our *SystemCalculator* class.

In [3]:
dico = dict()
dico['dft'] = dict(ixc='LDA')
dico['output'] = dict(orbitals='binary')

The wavefunctions will be present at the end of calculation, by indicating the value of the key *orbitals* in the output dictionary.

In [4]:
#Run the code with the LDA prefix, using a python dictionary (dico) and the file 'posinp.xyz' for the atomic positions.
study.run(name="LDA",input=dico,posinp='posinp.xyz')

Add a "posinp" key with the value "posinp.xyz" into the dictionary input
Creating from a dictionary the yaml input file "LDA.yaml"
Executing command:  /local/deutsch/Forge/BigDFT/build-mpif90/install/bin/bigdft -n LDA


<BigDFT.Logfiles.Logfile instance at 0x7f5e96fba518>

When using a naming scheme, the output files are placed in a directory called  **data-LDA**. In our LDA example, the wavefunctions of the system can thus be found in the **data-LDA** directory:

In [5]:
!ls data-LDA

time-LDA.yaml			  wavefunction-k001-NR.bin.b000004
wavefunction-k001-NR.bin.b000001  wavefunction-k001-NR.bin.b000005
wavefunction-k001-NR.bin.b000002  wavefunction-rhoij.bin
wavefunction-k001-NR.bin.b000003


Here 001 means the first K-point (meaningless in this case), N stands for non spin-polarized, R for real part and the remaining number is the orbital ID. Post-processing of these files is done in the fourth tutorial.

In the same spirit, another calculation can be done with different parameters. Imagine we want to perform a Hartree-Fock calculation. In BigDFT, this can be done by putting the ixc input variable to HF (or 100). So, we use the same dictionary changing only the *dft* key.
We add also a *posinp* key to indicate what atomic positions file we use (this was done when we indicate posinp='posinp.xyz' into the *run* method).

In [6]:
dico['dft']['ixc'] = 'HF'
dico['posinp'] = 'posinp.xyz'
study.run(name="HF",input=dico) #Run the code with the name scheme HF

Creating from a dictionary the yaml input file "HF.yaml"
Executing command:  /local/deutsch/Forge/BigDFT/build-mpif90/install/bin/bigdft -n HF


<BigDFT.Logfiles.Logfile instance at 0x7f5e9703cab8>

This time, an error will occur which is specified at the end of the file [log-HF.yaml](./log-HF.yaml)
and also in [debug/bigdft-err-0.yaml](./debug/bigdft-err-0.yaml):

This is because the pseudopotential is assigned by default in the code only for LDA and PBE XC approximations. You can find <a href="http://bigdft.org/images/f/f5/Psppar.N">here</a> the pseudopotential which is taken by default in the LDA run. Put it in a psppar.N file, and run the calculation.

In [7]:
study.run(name="HF") #Run the code with the name scheme

Executing command:  /local/deutsch/Forge/BigDFT/build-mpif90/install/bin/bigdft -n HF


<BigDFT.Logfiles.Logfile instance at 0x7f5ec84706c8>

When possible, care should be taken in choosing a pseudopotential which has been generated with the same XC approximation used. Unfortunately, at present HGH data are only available for semilocal functionals. For example, the same exercise as follows could have been done with Hybrid XC functionals, like for example PBE0 (ixc=-406). See the <a href="http://bigdft.org/Wiki/index.php?title=XC_codes">XC codes</a> for a list of the supported XC functionals. Data of the calculations can be analysed.

In [8]:
dico['dft']['ixc'] = 'PBE0'
study.run(name="PBE0",input=dico) #Run the code with the name scheme PBE0

Creating from a dictionary the yaml input file "PBE0.yaml"
Executing command:  /local/deutsch/Forge/BigDFT/build-mpif90/install/bin/bigdft -n PBE0


<BigDFT.Logfiles.Logfile instance at 0x7f5ed0940320>

For each run, you can see a string as <BigDFT.Logfiles.Logfile instance at 0x....>. The *run* method returns an instance of Logfile which contains all useful information about the corresponding calculation. We will see in another [tutorial](./Tutorial-N2-solution.ipynb) how to use it.

<h2>Exercise</h2>

Compare the values of the HOMO and HOMO-1 eigenvalues for the LDA and the HF run.
Change the values of the hgrid and crmult to find the converged values.
Note that, both in the LDA and in the HF calculation, a norm-conserving PSP is used.
The results can be compared to all-electron calculations, done with different basis sets, from references (units are eV) 
(1) S.&nbsp;Hamel <i>et&nbsp;al.</i> J. Electron Spectrospcopy and Related Phenomena 123 (2002) 345-363 and (2) P.&nbsp;Politzer, F.&nbsp;Abu-Awwad, Theor. Chem. Acc. (1998), 99, 83-87:

<table>
  <tr> <td></td>                      <td>LDA(1)</td>  <td>HF(1)</td>   <td>HF(2)</td>   <td>(Exp.)</td></tr>
  <tr> <td>3&sigma;<sub>g</sub></td>  <td>10.36</td>   <td>17.25</td>   <td>17.31</td>   <td>(15.60)</td></tr>
 <tr> <td>1&pi;<sub>u</sub></td>      <td>11.84</td>   <td>16.71</td>   <td>17.02</td>   <td>(16.98)</td></tr>
 <tr> <td>2&sigma;<sub>u</sub></td>   <td>13.41</td>   <td>21.25</td>   <td>21.08</td>   <td>(18.78)</td></tr>
</table>
 
The results depends, of course, on the precision chosen for the calculation, and of the presence of the pseudopotential.
As it is well-known, the pseudopotential appoximation is however much less severe than the approximation induced by typical XC functionals. We might see that, even in the HF case, the presence of a LDA-based pseudopotential (of rather good quality) does not alter so much the results. Here you can find the values from BigDFT calculation using a very good precision (*hgrid=0.3*, *crmult=7.0*). 
Note that 1 ha=27.21138386 eV.
 
<table>
  <tr> <td></td>                        <td>LDA</td>    <td>HF</td></tr>
  <tr> <td> 3&sigma;<sub>g</sub></td>   <td>10.40</td>  <td>17.32</td></tr>
  <tr> <td> 1&pi;<sub>u</sub></td>      <td>11.75</td>  <td>16.62</td></tr>
  <tr> <td> 2&sigma;<sub>u</sub></td>   <td>13.52</td>  <td>21.30</td></tr>
</table>

How much do these values differ from the calculation with default parameters? Do they converge to a given value?
What is the *correlation* for the N2 molecule in (PSP) LDA?

See here the [solution](./Tutorial-N2-solution.ipynb).